In [143]:
#CALL Libraries
import os
import pandas as pd
from lxml import etree
import re
from datetime import datetime
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
from collections import defaultdict
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pytz


# 2023 CHART

In [146]:
####EXTRACT AI - EARNING CALLS FROM STREET EVENT

# Define the parent folder path
parent_folder_path = '/Users/dali/Downloads/Archive'
total=0
num_pattern_wordsT=0
num_pattern_wordsqaT=0
# Create an empty list to store the extracted data
data_list = []

#pattern2 = re.compile(r'AI\b|artificial intelligence\b|Artificial Intelligence\b|\bai\b|\bai-\b',re.IGNORECASE)
pattern2 = re.compile(r'(\bAI\b)|(\bai\b)',re.IGNORECASE)

# Name of folders
years = ['2023','current']

# Iterate over the subdirectories (years) in the parent folder
for year in years:
    year_folder_path = parent_folder_path+'/'+year
    print(year_folder_path)

    # Iterate over the XML files in the year folder
    for filename in os.listdir(year_folder_path):
        count=0
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(year_folder_path, filename)
            # Read the XML content from the file
            xml_tree = etree.parse(xml_file_path)
            root = xml_tree.getroot()
            body_text = root.findtext('.//Body')

            # Extract Event ID
            event_id = root.attrib.get('Id', '')
            start_date = root.findtext('.//startDate')

            cusip = root.findtext('.//CUSIP')
            # Extract Event Type Name
            event_type_name = root.attrib.get('eventTypeName', '')

            ######Company Data########
            company_name = root.findtext('.//companyName')
            company_ticker = root.findtext('.//companyTicker')
            company_id = root.findtext('.//companyId')

            
            ######Presentation########
            body_element = root.find('.//Body')
            if body_element is not None and body_element.text:
                body_text = body_element.text.strip()

                pattern = re.compile(r'(?:Presentation|Transcript)\s+-{1,}\s+(.*?)\s+(?:==|=]{2,}|Questions and Answers|q a|]]>)', re.DOTALL)
                # Find the presentation or transcript match
                presentation_match = pattern.search(body_text)
                # Extract the presentation or transcript text
                presentation = presentation_match.group(1).strip() if presentation_match else ""
                # Count the number of words in the presentation 
                # Count the number of words related to the pattern
                num_pattern_words = len(re.findall(pattern2, presentation))
                num_pattern_wordsT +=len(re.findall(pattern2, presentation))
                #print(num_pattern_words)
            

            ######Q&A########
            body_elementqa = root.find('.//Body')
            if body_elementqa is not None and body_elementqa.text:
                body_textqa = body_elementqa.text.strip()

                patternqa = re.compile(r'==\s*(?:Questions\s+and\s+Answers|q\s*[na]\s*a|qa|qna)(.*)', re.DOTALL)
                #print(patternqa)
            # Find the presentation or transcript match
                qa_match = patternqa.search(body_textqa)
                #print(qa_match)
                # Extract the presentation or transcript text
                qa = qa_match.group(1).strip() if qa_match  else ""
                num_pattern_wordsqa = len(re.findall(pattern2, qa))
                num_pattern_wordsqaT += len(re.findall(pattern2, qa))

               # if(qa_match is None):
                #    print("qa",filename,len(qa.split()))
            
            total=num_pattern_wordsqa+num_pattern_words
           
            #total+=num_pattern_words
   
            
            
            
            # Check if the presentation contains the word "AI"
            contains_ai = bool(pattern2.search(presentation)) or bool(pattern2.search(qa))
            
            match = pattern.search(presentation)
            #if match:
            #    word = match.group()
            #    print(word, filename)
                
                 # Append the extracted data to the data_list
            data_list.append({
                "Event ID": event_id,
                'eventTypeName':event_type_name,
                "Start Date": start_date,
                "CUSIP": cusip,
                "Company ID": company_id,
                "Company Name": company_name,
                "Company Ticker": company_ticker,
                "AI_term": contains_ai,
                "AI_term_P": total
            })

# Create a DataFrame from the extracted data
df = pd.DataFrame(data_list)

print('done')
print('AI from presentations',num_pattern_wordsT)
print('AI from QA',num_pattern_wordsqaT)

/Users/dali/Downloads/Archive/2023
/Users/dali/Downloads/Archive/current
done
AI from presentations 8791
AI from QA 12105
AI from both 0


In [139]:
# Convert the 'Start Date' column to datetime format
df['Start Date'] = pd.to_datetime(df['Start Date'], format='%d-%b-%y %I:%M%p %Z')

# Define the start and end dates in the same time zone as 'Start Date' column
start_date = pd.to_datetime('2023-03-15').tz_localize('GMT')
end_date = pd.to_datetime('2023-05-25').tz_localize('GMT')

# Filter the DataFrame based on conditions
filtered_df = df[
    (df['eventTypeName'] == 'Earning Conference Call/Presentation') &
    (df['Start Date'].between(start_date, end_date))
]

# Define the output Excel file path
output_file_path = '/Users/dali/Documents/task 1/SE_EC_AI_Mar_May2023.csv'

# Save the DataFrame to an csv file
filtered_df.to_csv(output_file_path, index=False)

In [147]:
sp500_data = pd.read_csv("/Users/dali/Documents/task 1/sp500_by_sectors.csv")
SE_23_data = pd.read_csv("/Users/dali/Documents/task 1/SE_EC_AI_Mar_May2023.csv")

In [150]:
# Extract the first 6 digits of CUSIP and ncusip columns
SE_23_data['CUSIP_6'] = SE_23_data['CUSIP'].apply(lambda x: str(x)[:6])
sp500_data['ncusip_6'] = sp500_data['ncusip'].apply(lambda x: str(x)[:6])


# Merge the two datasets based on the first 6 digits of CUSIP and ncusip columns
merged_data = pd.merge( sp500_data, SE_23_data, left_on='ncusip_6', right_on='CUSIP_6', how='inner')


# Drop the unnecessary columns
merged_data.drop(['CUSIP_6', 'ncusip_6','start', 'ending',
                  'namedt', 'nameenddt','Unnamed: 0','permno'], axis=1, inplace=True)


# Drop duplicate entries based on unique cusips in sp500_data
merged_data.drop_duplicates(subset=['CUSIP'], inplace=True)


# Save the merged data to a new file
merged_data.to_csv('/Users/dali/Documents/task 1/merged_data_SP_EC.csv', index=False)#RETURN ALL SP EARNING CALLS 


num_rows = merged_data.shape[0]
print('total sp500 earning calls:',num_rows)



total sp500 earning calls: 490


In [149]:
sector_counts = merged_data.groupby('Sector_Name')['AI_term'].value_counts()
sector_percentages = sector_counts.groupby(level=0).apply(lambda x: x / x.sum() * 100)

# Access the percentage of True values for each sector
true_percentages = sector_percentages.loc[:, True]

# Print the sector name and the corresponding percentage of True values
for sector, percentage in true_percentages.iteritems():
    rounded_percentage = round(percentage, 2)
    print(f"Sector: {sector}, Percentage of True values: {rounded_percentage}%")


Sector: Communication Services, Percentage of True values: 75.0%
Sector: Consumer Discretionary, Percentage of True values: 16.42%
Sector: Consumer Staples, Percentage of True values: 3.7%
Sector: Financials, Percentage of True values: 13.33%
Sector: Health Care, Percentage of True values: 13.33%
Sector: Industrials, Percentage of True values: 15.94%
Sector: Information Technology, Percentage of True values: 63.64%
Sector: Materials, Percentage of True values: 3.23%
Sector: Real Estate, Percentage of True values: 16.13%


/var/folders/qw/kqdccjfj4dl03z7q6lwkyjt80000gq/T/ipykernel_7200/70446688.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sector_percentages = sector_counts.groupby(level=0).apply(lambda x: x / x.sum() * 100)
/var/folders/qw/kqdccjfj4dl03z7q6lwkyjt80000gq/T/ipykernel_7200/70446688.py:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for sector, percentage in true_percentages.iteritems():
